# Simulación de Eventos Discretos - TP1

TODO: explicar modelo conceptual

Hace falta compilar el simulador con los atómicos nuevos:

In [ ]:
%%bash 

cd src/

make

## Atómico Producto

In [ ]:
%%bash

cat model/test_product.ma

In [ ]:
%%bash

cat model/test_product.ev

In [ ]:
%%bash

src/bin/cd++ -m model/test_product.ma \
             -t 10:00:00:000 \
             -l test_product.log \
             -o test_product.out \
             -e model/test_product.ev

In [ ]:
%%bash

cat model.out

## Atomico Mercado

In [ ]:
%%bash

cat model/test_market.ma

In [ ]:
%%bash

cat model/test_market.ev

In [1]:
%%bash

src/bin/cd++ -m model/test_market.ma \
             -t 05:00:00:000 \
             -l test_market.log \
             -o test_market.out \

PCD++: A Tool to Implement n-Dimensional Cell-DEVS models
Version 3.0 - March 2003
Troccoli A., Rodriguez D., Wainer G., Barylko A., Beyoglonian J., Lopez A.
-----------------------------------------------------------------------------
PCD++ Extended States: An extended and improved version of CD++ for Cell-DEVS
Version 4.1.2 - December 2018
Santi L., Castro, R., Pimás, J.
-----------------------------------------------------------------------------
Discrete Event Simulation Lab
Departamento de Computación
Facultad de Ciencias Exactas y Naturales
Universidad de Buenos Aires, Argentina
-----------------------------------------------------------------------------
Compiled for standalone simulation


Loading models from model/test_market.ma
Loading events from 
Running parallel simulation. Reading models partition from 
Model partition details output to: /dev/null*
Message log: test_market.log
Output to: test_market.out
Tolerance set to: 1e-08
Configuration to show real numbers: Width = 1

Exception MException thrown!
Description: Data not found!!!
R(etry), I(gnore), P(anic)?
Thrown in: 
	File: ini.cpp - Method: definition - Line: 55
Description: 
	p0/initialVolume Not Found!


In [2]:
%%bash

cat test_market.out

00:00:00:000:0 supply_p0   1000.00000
00:00:00:000:0 supply_p1   1000.00000
01:00:00:000:0 supply_p0   1000.00000
01:00:00:000:0 supply_p1   1000.00000
02:00:00:000:0 supply_p0   1000.00000
02:00:00:000:0 supply_p1   1000.00000
03:00:00:000:0 supply_p0   1000.00000
03:00:00:000:0 supply_p1   1000.00000
04:00:00:000:0 supply_p0   1000.00000
04:00:00:000:0 supply_p1   1000.00000
05:00:00:000:0 supply_p0   1000.00000
05:00:00:000:0 supply_p1   1000.00000


In [4]:
%%bash

cat test_market.log01

0 / L / I / 00:00:00:000:0 / top(05) / market(01)
0 / L / X / 00:00:00:000:0 / top(05) / supply_c0 / [1800, 3100] / market(01)
0 / L / X / 00:00:00:000:0 / top(05) / demand_p0 /   1000.00000 / market(01)
0 / L / X / 00:00:00:000:0 / top(05) / demand_p1 /   1000.00000 / market(01)
0 / L / * / 00:00:00:000:0 / top(05) / market(01)
0 / L / @ / 00:00:00:000:0 / top(05) / market(01)
0 / L / * / 00:00:00:000:0 / top(05) / market(01)
0 / L / X / 00:00:00:000:0 / top(05) / supply_c0 / [1000, 1000] / market(01)
0 / L / * / 00:00:00:000:0 / top(05) / market(01)
0 / L / @ / 00:00:00:000:0 / top(05) / market(01)
0 / L / * / 00:00:00:000:0 / top(05) / market(01)
0 / L / X / 00:00:00:000:0 / top(05) / supply_c0 / [1000, 1000] / market(01)
0 / L / * / 00:00:00:000:0 / top(05) / market(01)
0 / L / @ / 00:00:00:000:0 / top(05) / market(01)
0 / L / * / 00:00:00:000:0 / top(05) / market(01)
0 / L / X / 01:00:00:000:0 / top(05) / demand_p0 /   1000.00000 / market(01)
0 / L / X / 01:00:00:000:0 / top(05) /

## Atómico Pais

In [ ]:
%%bash

cat model/test_country.ma

In [ ]:
%%bash

cat model/test_country.ev

In [1]:
%%bash

src/bin/cd++ -m model/test_country.ma \
             -t 05:00:00:000 \
             -l test_country.log \
             -o test_country.out \
             -e model/test_country.ev

PCD++: A Tool to Implement n-Dimensional Cell-DEVS models
Version 3.0 - March 2003
Troccoli A., Rodriguez D., Wainer G., Barylko A., Beyoglonian J., Lopez A.
-----------------------------------------------------------------------------
PCD++ Extended States: An extended and improved version of CD++ for Cell-DEVS
Version 4.1.2 - December 2018
Santi L., Castro, R., Pimás, J.
-----------------------------------------------------------------------------
Discrete Event Simulation Lab
Departamento de Computación
Facultad de Ciencias Exactas y Naturales
Universidad de Buenos Aires, Argentina
-----------------------------------------------------------------------------
Compiled for standalone simulation


Loading models from model/test_country.ma
Loading events from model/test_country.ev
Running parallel simulation. Reading models partition from 
Model partition details output to: /dev/null*
Message log: test_country.log
Output to: test_country.out
Tolerance set to: 1e-08
Configuration to show

In [2]:
%%bash

cat test_country.out

00:00:00:000:0 out [1800, 3100, 0, 0, 8000]
00:00:00:000:0 out [2000, 3000, 0, 0, 8500]
01:00:00:000:0 out [2100, 3200, 100, 200, 8600]
02:00:00:000:0 out [2200, 3100, 200, 300, 8400]
03:00:00:000:0 out [2500, 3050, 700, 360, 8000]
04:00:00:000:0 out [2800, 3000, 600, 410, 7500]


In [5]:
%%bash

src/bin/cd++ -m model/simulation.ma \
             -t 05:00:00:000 \
             -l simulation.log \
             -o simulation.out

PCD++: A Tool to Implement n-Dimensional Cell-DEVS models
Version 3.0 - March 2003
Troccoli A., Rodriguez D., Wainer G., Barylko A., Beyoglonian J., Lopez A.
-----------------------------------------------------------------------------
PCD++ Extended States: An extended and improved version of CD++ for Cell-DEVS
Version 4.1.2 - December 2018
Santi L., Castro, R., Pimás, J.
-----------------------------------------------------------------------------
Discrete Event Simulation Lab
Departamento de Computación
Facultad de Ciencias Exactas y Naturales
Universidad de Buenos Aires, Argentina
-----------------------------------------------------------------------------
Compiled for standalone simulation


Loading models from model/simulation.ma
Loading events from 
Running parallel simulation. Reading models partition from 
Model partition details output to: /dev/null*
Message log: simulation.log
Output to: simulation.out
Tolerance set to: 1e-08
Configuration to show real numbers: Width = 12 -

*** stack smashing detected ***: terminated
bash: line 5: 1526217 Aborted                 (core dumped) src/bin/cd++ -m model/simulation.ma -t 05:00:00:000 -l simulation.log -o simulation.out


CalledProcessError: Command 'b'\nsrc/bin/cd++ -m model/simulation.ma \\\n             -t 05:00:00:000 \\\n             -l simulation.log \\\n             -o simulation.out\n'' returned non-zero exit status 134.